## Setup

In [ ]:
# Necessary imports

from datetime import datetime, timedelta
import os
import json
import glob
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import pandas as pd
from scipy.stats import iqr
import shutil
import gdown
import zipfile

%matplotlib inline

In [ ]:
start_time = datetime.now()

In [ ]:
#gdown.download(url="https://drive.google.com/file/d/1PGqqp09sCKfmFE1oewx0m7I2Xdl9eu8e/view?usp=sharing", fuzzy=True)
#with zipfile.ZipFile("data_2024-02-17.zip", "r") as archive:
#    archive.extractall(".")

In [ ]:
# Define useful variables
path_data = os.path.join(os.getcwd(), "data/")
print(f"path_data: {path_data}")
path_labels = os.path.join(path_data, "derivatives", "labels")
path_qc = os.path.join(path_data, "qc")
subjects = [os.path.basename(subject_path) for subject_path in sorted(glob.glob(os.path.join(path_data, "sub-*")))]
print(f"subjects: {subjects}")

# Create output folder
path_results = os.path.join(path_data, "derivatives", "results")
os.makedirs(path_results, exist_ok=True)

In [ ]:
# Remove SpinozaV6 for now
for subject in subjects:
    if subject[-1] == "4":
        subjects.remove(subject)
print(subjects)

In [ ]:
## MP2RAGE segmentation

In [ ]:
# Run segmentation on MP2RAGE scan

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "anat"))
    fname_manual_seg = os.path.join(path_labels, subject, "anat", f"{subject}_UNIT1-SC_seg.nii.gz")
    if os.path.exists(fname_manual_seg):
        # Manual segmentation already exists. Copy it to local folder
        print(f"{subject}: Manual segmentation found\n")
        shutil.copyfile(fname_manual_seg, f"{subject}_UNIT1_seg.nii.gz")
        # Generate QC report to make sure the manual segmentation is correct
        !sct_qc -i {subject}_UNIT1.nii.gz -s {subject}_UNIT1_seg.nii.gz -p sct_deepseg_sc -qc {path_qc} -qc-subject {subject}
    else:
        # Manual segmentation does not exist. Run automatic segmentation.
        print(f"{subject}: Manual segmentation not found")
        !sct_deepseg_sc -i "{subject}_UNIT1.nii.gz" -c t1 -qc "{path_qc}"

## Process fmap/TFL (flip angle maps)

In [ ]:
# Register TFL flip angle maps to the MP2RAGE scan ⏳

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "fmap"))
    !sct_register_multimodal -i {subject}_acq-anat_TB1TFL.nii.gz -d ../anat/{subject}_UNIT1.nii.gz -dseg ../anat/{subject}_UNIT1_seg.nii.gz -param step=1,type=im,algo=slicereg,metric=CC -qc "{path_qc}"

In [ ]:
# Warping spinal cord segmentation and vertebral level to each flip angle map

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "fmap"))
    !sct_apply_transfo -i ../anat/{subject}_UNIT1_seg.nii.gz -d {subject}_acq-anat_TB1TFL.nii.gz -w warp_{subject}_UNIT12{subject}_acq-anat_TB1TFL.nii.gz -x linear -o {subject}_acq-anat_TB1TFL_seg.nii.gz

In [ ]:
# Convert the flip angle maps to B1+ efficiency maps [nT/V] (inspired by code from Kyle Gilbert)
# The approach consists in calculating the B1+ efficiency using a 1ms, pi-pulse at the acquisition voltage,
# then scale the efficiency by the ratio of the measured flip angle to the requested flip angle in the pulse sequence.

GAMMA = 2.675e8;  # [rad / (s T)]
requested_fa = 90  # saturation flip angle -- hard-coded in sequence

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "fmap"))
    # Fetch the reference voltage from the JSON sidecar to the TFL B1map sequence
    with open(f"{subject}_acq-famp_TB1TFL.json", "r") as f:
        metadata = json.load(f)
        ref_voltage = metadata.get("TxRefAmp", "N/A")
        if (ref_voltage == "N/A"):
            ref_token = "N/A"
            for token in metadata.get("SeriesDescription", "N/A").split("_"):
                if token.startswith("RefV"): ref_token = token
            ref_voltage = float(ref_token[4:-1])
        print(f"ref_voltage [V]: {ref_voltage} ({subject}_acq-famp_TB1TFL)")

    # Open flip angle map with nibabel
    nii = nib.load(f"{subject}_acq-famp_TB1TFL.nii.gz")
    acquired_fa = nii.get_fdata()

    # Siemens maps are in units of flip angle * 10 (in degrees)
    acquired_fa = acquired_fa / 10

    # Account for the power loss between the coil and the socket. That number was given by Siemens.
    voltage_at_socket = ref_voltage * 10 ** -0.095

    # Compute B1 map in [T/V]
    b1_map = (acquired_fa / requested_fa) * (np.pi / (GAMMA * 1e-3 * voltage_at_socket))

    # Convert to [nT/V]
    b1_map = b1_map * 1e9

    # Save as NIfTI file
    nii_b1 = nib.Nifti1Image(b1_map, nii.affine, nii.header)
    nib.save(nii_b1, f"{subject}_TB1map.nii.gz")

In [ ]:
# Extract B1+ value along the spinal cord between levels C3 and T2 (included)

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "fmap"))
    fname_result_b1plus = os.path.join(path_results, f"{subject}_TB1map.csv")
    !sct_extract_metric -i {subject}_TB1map.nii.gz -f {subject}_acq-anat_TB1TFL_seg.nii.gz -method wa -perslice 1 -o "{fname_result_b1plus}"

In [ ]:
def signal_extractor_from_csv(csv_filename):
  # Load the CSV file into a Pandas DataFrame
  dataframe = pd.read_csv(csv_filename, index_col='Slice (I->S)')
  # Convert all the strings (123.4242, etc) of the WA column into actual numerical values
  dataframe['WA()'] = pd.to_numeric(dataframe['WA()'], errors='coerce')
  WA_matrix = dataframe['WA()'].to_numpy()
  return WA_matrix

b1_iqrs = []
TFL_B1_nTpV_along_cord_list = []

for subject in subjects:
    TFL_B1_nTpV_along_cord = signal_extractor_from_csv(os.path.join(path_results, subject + "_TB1map.csv"))
    TFL_B1_nTpV_along_cord = TFL_B1_nTpV_along_cord[~np.isnan(TFL_B1_nTpV_along_cord)] # remove NaNs
    TFL_B1_nTpV_along_cord_list.append(TFL_B1_nTpV_along_cord)
    plt.plot(TFL_B1_nTpV_along_cord)
    TFL_B1_nTpV_along_cord_mean = np.round(np.mean(TFL_B1_nTpV_along_cord))
    std = np.std(TFL_B1_nTpV_along_cord)
    titlestring= f"B1+ efficiency [nT/V] along the cord in the warped mask. Mean value: {TFL_B1_nTpV_along_cord_mean} Std: {std}"
    b1_iqrs.append(iqr(TFL_B1_nTpV_along_cord))
    plt.title(titlestring)
    plt.xlabel('Slice along the I-->S direction')
    plt.ylabel('nT/V')
    plt.show()  



In [ ]:
sites = ["MGH", "MNI", "NYU"]
subject_names = ["Subject D", "Subject L", "Subject R"]

fig = plt.figure()
gs = fig.add_gridspec(1, 3, wspace=0)
axs = gs.subplots(sharex=True, sharey=True)
fig.set_size_inches(16, 6)

j = 0
for i in range(len(sites)): 
    axs[i].plot(TFL_B1_nTpV_along_cord_list[j])
    axs[i].plot(TFL_B1_nTpV_along_cord_list[j+1])
    axs[i].plot(TFL_B1_nTpV_along_cord_list[j+2])
    axs[i].set_title(sites[i])
    j = j + 3

axs[i].legend(subject_names,loc="upper right")


for ax in axs.flat:
    ax.set(xlabel='Slice along the I-->S direction', ylabel='nT/V')

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()

plt.show()

In [ ]:
series = [b1_iqrs[i::3]for i in range(3)]
hline_x = np.array([0, 1, 2])
hline_width = 0.25

fig, ax = plt.subplots()
for subject_name, subject_series in zip(subject_names, series):
    ax.scatter(sites, subject_series, label=subject_name)
plt.hlines([np.mean(b1_iqrs[i:i+3]) for i in range(0, len(b1_iqrs), 3)], hline_x - hline_width/2, hline_x + hline_width/2, color="black", label="Across subj. mean")
ax.legend()
ax.set_ylim(2, 20)
ax.set_title("IQR B1+: IQR across slices within SC ROI")
plt.show()

In [ ]:
# Indicate duration of data processing

end_time = datetime.now()
total_time = (end_time - start_time).total_seconds()

# Convert seconds to a timedelta object
total_time_delta = timedelta(seconds=total_time)

# Format the timedelta object to a string
formatted_time = str(total_time_delta)

# Pad the string representation if less than an hour
formatted_time = formatted_time.rjust(8, '0')

print(f"Total Runtime [hour:min:sec]: {formatted_time}")